# ⚙️ Configuration Guide

**Before running this notebook, review and adjust the settings below based on your environment.**

## Quick Setup

| Setting | Google Colab | Local Machine |
|---------|--------------|---------------|
| `USE_COLAB` | `True` | `False` |
| `USE_WANDB` | `True` (optional) | `True` (optional) |
| `USE_DRIVE` | `True` (optional) | `False` |

## Environment Options

### 1. Execution Environment
- **Google Colab (Recommended)**: Click the "Open in Colab" badge, set `USE_COLAB = True`
- **Local Machine**: Clone repo, install requirements, set `USE_COLAB = False`

### 2. Experiment Tracking (Weights & Biases)
- **Enable**: Set `USE_WANDB = True`, you'll be prompted to login
- **Disable**: Set `USE_WANDB = False` (no account needed)

### 3. Model Storage
- **Google Drive** (Colab only): Set `USE_DRIVE = True`
- **Local Folder**: Set `USE_DRIVE = False` (saves to `./DermaMNIST_Study/`)

### 4. GPU
- **Colab**: Runtime → Change runtime type → GPU (T4)
- **Local**: Requires NVIDIA GPU with CUDA

---
**Run the configuration cell below, then proceed with the notebook.**

In [ ]:
# ============================================================================
# Note: Configuration variables (USE_COLAB, USE_WANDB, USE_DRIVE, SEED, PROJECT_NAME)
#       are defined in the Configuration cell above.

# INSTALLATION
!pip install -q medmnist wandb

# ============================================================================
# IMPORTS
import os
import random
import gc
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Deep Learning Imports
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, DenseNet121  # Lightweight models for small images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K

# Metrics & Data
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix)
from sklearn.utils import class_weight
from medmnist import DermaMNIST

# ============================================================================
# CONDITIONAL IMPORTS (based on configuration)

# Weights & Biases
if USE_WANDB:
    try:
        import wandb
        from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

        # ┌─────────────────────────────────────────────────────────────────┐
        # │ W&B LOGIN - Load API key from Colab Secrets                     │
        # └─────────────────────────────────────────────────────────────────┘
        try:
            from google.colab import userdata
            WANDB_API_KEY = userdata.get('WANDB_API_KEY')
            wandb.login(key=WANDB_API_KEY)
            print("✅ W&B logged in via Colab Secrets")
        except Exception as e:
            print(f"⚠️ Could not auto-login to W&B: {e}")
            print("   You will be prompted to login on first run.")

    except ImportError:
        print("⚠️ W&B not installed. Setting USE_WANDB = False")
        USE_WANDB = False
        wandb = None
else:
    wandb = None
    print("ℹ️ W&B tracking disabled (USE_WANDB = False)")

# Google Colab
if USE_COLAB:
    try:
        from google.colab import drive
        print("✅ Running in Google Colab")
    except ImportError:
        print("⚠️ Not in Colab environment. Setting USE_COLAB = False")
        USE_COLAB = False

# ============================================================================
# REPRODUCIBILITY
def set_seeds(seed=SEED):
    """Set random seeds for reproducibility across all libraries."""
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seeds()
print(f"✅ Random seed set to {SEED}")

# ============================================================================
# GPU CHECK
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detected: {gpus[0].name}")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("⚠️ NO GPU DETECTED - Training will be slow!")
    print("   Colab: Runtime → Change runtime type → GPU")
    print("   Local: Ensure CUDA and cuDNN are installed")

print(f"✅ TensorFlow version: {tf.__version__}")

# ============================================================================
# STORAGE SETUP
if USE_COLAB and USE_DRIVE:
    drive.mount('/content/drive')
    BASE_PATH = f'/content/drive/MyDrive/{PROJECT_NAME}'
    print(f"✅ Google Drive mounted")
else:
    BASE_PATH = f'./{PROJECT_NAME}'
    print(f"ℹ️ Using local storage")

DIR_PHASE_1 = os.path.join(BASE_PATH, 'Phase1_Baselines')
DIR_PHASE_2 = os.path.join(BASE_PATH, 'Phase2_Tuning')
DIR_PHASE_3 = os.path.join(BASE_PATH, 'Phase3_Final')

for folder in [DIR_PHASE_1, DIR_PHASE_2, DIR_PHASE_3]:
    os.makedirs(folder, exist_ok=True)

print(f"\n📁 Project: {PROJECT_NAME}")
print(f"📁 Storage paths:")
print(f"   Phase 1: {DIR_PHASE_1}")
print(f"   Phase 2: {DIR_PHASE_2}")
print(f"   Phase 3: {DIR_PHASE_3}")

# ============================================================================
# UTILITY FUNCTIONS
def save_to_drive(experiment_name, target_folder):
    """
    Save model checkpoint to the specified folder.

    Args:
        experiment_name (str): Name of the experiment (without .keras extension)
        target_folder (str): Destination folder path
    """
    filename = f"{experiment_name}.keras"
    source = filename
    destination = os.path.join(target_folder, filename)

    if os.path.exists(source):
        shutil.copy(source, destination)
        print(f"💾 Model saved: {destination}")
    else:
        print(f"⚠️ Model file not found: {source}")

print("\n" + "=" * 60)
print("✅ SETUP COMPLETE")
print("=" * 60)
print("""
""")

# Block 1: Environment Setup

Initializes the runtime environment: installs dependencies, configures GPU memory,
sets random seeds for reproducibility, and creates storage directories for model checkpoints.

In [ ]:
# ============================================================================
# Note: Configuration variables (USE_COLAB, USE_WANDB, USE_DRIVE, SEED, PROJECT_NAME)
#       are defined in the Configuration cell above.

# INSTALLATION
!pip install -q medmnist wandb

# ============================================================================
# IMPORTS
import os
import random
import gc
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Deep Learning Imports
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0  # Lightweight models for small images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K

# Metrics & Data
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix)
from sklearn.utils import class_weight
from medmnist import DermaMNIST

# ============================================================================
# CONDITIONAL IMPORTS (based on configuration)

# Weights & Biases
if USE_WANDB:
    try:
        import wandb
        from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

        # ┌─────────────────────────────────────────────────────────────────┐
        # │ W&B LOGIN - Load API key from Colab Secrets                     │
        # └─────────────────────────────────────────────────────────────────┘
        try:
            from google.colab import userdata
            WANDB_API_KEY = userdata.get('WANDB_API_KEY')
            wandb.login(key=WANDB_API_KEY)
            print("✅ W&B logged in via Colab Secrets")
        except Exception as e:
            print(f"⚠️ Could not auto-login to W&B: {e}")
            print("   You will be prompted to login on first run.")

    except ImportError:
        print("⚠️ W&B not installed. Setting USE_WANDB = False")
        USE_WANDB = False
        wandb = None
else:
    wandb = None
    print("ℹ️ W&B tracking disabled (USE_WANDB = False)")

# Google Colab
if USE_COLAB:
    try:
        from google.colab import drive
        print("✅ Running in Google Colab")
    except ImportError:
        print("⚠️ Not in Colab environment. Setting USE_COLAB = False")
        USE_COLAB = False

# ============================================================================
# REPRODUCIBILITY
def set_seeds(seed=SEED):
    """Set random seeds for reproducibility across all libraries."""
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seeds()
print(f"✅ Random seed set to {SEED}")

# ============================================================================
# GPU CHECK
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detected: {gpus[0].name}")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("⚠️ NO GPU DETECTED - Training will be slow!")
    print("   Colab: Runtime → Change runtime type → GPU")
    print("   Local: Ensure CUDA and cuDNN are installed")

print(f"✅ TensorFlow version: {tf.__version__}")

# ============================================================================
# STORAGE SETUP
if USE_COLAB and USE_DRIVE:
    drive.mount('/content/drive')
    BASE_PATH = f'/content/drive/MyDrive/{PROJECT_NAME}'
    print(f"✅ Google Drive mounted")
else:
    BASE_PATH = f'./{PROJECT_NAME}'
    print(f"ℹ️ Using local storage")

DIR_PHASE_1 = os.path.join(BASE_PATH, 'Phase1_Baselines')
DIR_PHASE_2 = os.path.join(BASE_PATH, 'Phase2_Tuning')
DIR_PHASE_3 = os.path.join(BASE_PATH, 'Phase3_Final')

for folder in [DIR_PHASE_1, DIR_PHASE_2, DIR_PHASE_3]:
    os.makedirs(folder, exist_ok=True)

print(f"\n📁 Project: {PROJECT_NAME}")
print(f"📁 Storage paths:")
print(f"   Phase 1: {DIR_PHASE_1}")
print(f"   Phase 2: {DIR_PHASE_2}")
print(f"   Phase 3: {DIR_PHASE_3}")

# ============================================================================
# UTILITY FUNCTIONS
def save_to_drive(experiment_name, target_folder):
    """
    Save model checkpoint to the specified folder.

    Args:
        experiment_name (str): Name of the experiment (without .keras extension)
        target_folder (str): Destination folder path
    """
    filename = f"{experiment_name}.keras"
    source = filename
    destination = os.path.join(target_folder, filename)

    if os.path.exists(source):
        shutil.copy(source, destination)
        print(f"💾 Model saved: {destination}")
    else:
        print(f"⚠️ Model file not found: {source}")

print("\n" + "=" * 60)
print("✅ SETUP COMPLETE")
print("=" * 60)
print("""
""")

# Block 2: Data Loading & Exploratory Data Analysis

- **2a:** Explore raw data (7 original classes, pixel distributions)
- **2b:** Transform to binary classification (Malignant vs Benign)
- **2c:** Verify transformed data and compute class weights

**Dataset:** DermaMNIST (10,015 dermatoscopic images, 28×28×3 RGB)

**Binary Mapping:**
- **Malignant (1):** akiec, bcc, mel (classes 0, 1, 6)
- **Benign (0):** bkl, df, nv, vasc (classes 2, 3, 4, 5)

In [ ]:
# ============================================================================
# BLOCK 2a: EDA - RAW DATA EXPLORATION
# ============================================================================

print("="*60)
print("📊 EXPLORATORY DATA ANALYSIS - RAW DATA")
print("="*60)

raw_train = DermaMNIST(split='train', download=True, size=28)
raw_val = DermaMNIST(split='val', download=True, size=28)
raw_test = DermaMNIST(split='test', download=True, size=28)

print("\n1️⃣ DATASET OVERVIEW")
print("-"*40)
print(f"   Training samples:   {len(raw_train.imgs):,}")
print(f"   Validation samples: {len(raw_val.imgs):,}")
print(f"   Test samples:       {len(raw_test.imgs):,}")
print(f"   Total samples:      {len(raw_train.imgs) + len(raw_val.imgs) + len(raw_test.imgs):,}")

print("\n2️⃣ IMAGE PROPERTIES")
print("-"*40)
print(f"   Shape: {raw_train.imgs[0].shape}")
print(f"   Dtype: {raw_train.imgs[0].dtype}")
print(f"   Pixel range: [{raw_train.imgs.min()}, {raw_train.imgs.max()}]")

print("\n3️⃣ ORIGINAL LABEL DISTRIBUTION (7 Classes)")
print("-"*40)

class_names = {
    0: 'Actinic keratoses (akiec)',      # Pre-cancerous
    1: 'Basal cell carcinoma (bcc)',     # MALIGNANT - skin cancer
    2: 'Benign keratosis (bkl)',         # Benign
    3: 'Dermatofibroma (df)',            # Benign
    4: 'Melanoma (mel)',                 # MALIGNANT - most dangerous skin cancer
    5: 'Melanocytic nevi (nv)',          # Benign (common moles)
    6: 'Vascular lesions (vasc)'         # Benign
}

# Count each class
train_labels = raw_train.labels.flatten()
unique, counts = np.unique(train_labels, return_counts=True)

print(f"   {'Class':<35} {'Count':>7} {'Percent':>8}")
print(f"   {'-'*35} {'-'*7} {'-'*8}")
for cls, count in zip(unique, counts):
    pct = 100 * count / len(train_labels)
    print(f"   {class_names[cls]:<35} {count:>7,} {pct:>7.1f}%")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
ax1 = axes[0]
bars = ax1.bar(range(len(unique)), counts, color='steelblue', edgecolor='black')
ax1.set_xticks(range(len(unique)))
ax1.set_xticklabels([f"{i}\n{class_names[i].split('(')[1].replace(')', '')}" for i in unique], fontsize=9)
ax1.set_xlabel('Class')
ax1.set_ylabel('Count')
ax1.set_title('Original 7-Class Distribution (Training Set)')

# Add count labels on bars
for bar, count in zip(bars, counts):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
             f'{count:,}', ha='center', va='bottom', fontsize=9)

# Pie chart
ax2 = axes[1]
ax2.pie(counts, labels=[class_names[i].split('(')[1].replace(')', '') for i in unique],
        autopct='%1.1f%%', startangle=90)
ax2.set_title('Class Proportions (Training Set)')

plt.tight_layout()
plt.show()

print("\n4️⃣ SAMPLE IMAGES FROM EACH CLASS")
print("-"*40)

fig, axes = plt.subplots(2, 7, figsize=(16, 5))

for cls in range(7):
    cls_indices = np.where(train_labels == cls)[0]

    for row in range(2):
        idx = cls_indices[row]
        axes[row, cls].imshow(raw_train.imgs[idx])
        axes[row, cls].axis('off')
        if row == 0:
            short_name = class_names[cls].split('(')[1].replace(')', '')
            axes[row, cls].set_title(f'{short_name}\n(Class {cls})', fontsize=9)

plt.suptitle('Sample Images from Each Original Class', fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

print("\n5️⃣ BINARY MAPPING PREVIEW")
print("-"*40)
print("   Malignant (1): Classes 0, 1, 4")
print("      - 0: Actinic keratoses (akiec)   → Pre-cancerous")
print("      - 1: Basal cell carcinoma (bcc)  → Skin cancer")
print("      - 4: Melanoma (mel)              → Skin cancer (most dangerous)")
print("\n   Benign (0): Classes 2, 3, 5, 6")
print("      - 2: Benign keratosis (bkl)      → Benign growth")
print("      - 3: Dermatofibroma (df)         → Benign fibrous nodule")
print("      - 5: Melanocytic nevi (nv)       → Benign moles")
print("      - 6: Vascular lesions (vasc)     → Benign vascular")

# Calculate expected binary distribution
malignant_classes = [0, 1, 4]  # akiec, bcc, mel - CORRECTED!
malignant_count = sum(counts[i] for i in range(len(unique)) if unique[i] in malignant_classes)
benign_count = sum(counts[i] for i in range(len(unique)) if unique[i] not in malignant_classes)

print(f"\n   Expected after transformation:")
print(f"      Benign:    {benign_count:,} ({100*benign_count/len(train_labels):.1f}%)")
print(f"      Malignant: {malignant_count:,} ({100*malignant_count/len(train_labels):.1f}%)")
print(f"      Ratio:     {benign_count/malignant_count:.1f}:1 (Benign:Malignant)")

In [ ]:
# ============================================================================
# BLOCK 2b: DATA TRANSFORMATION - BINARY CLASSIFICATION (CORRECTED)
# ============================================================================

print("="*60)
print("🔄 DATA TRANSFORMATION - BINARY CLASSIFICATION")
print("="*60)

def transform_to_binary(labels):
    """
    Transform 7-class labels to binary (Benign=0, Malignant=1).

    CORRECTED MAPPING:
    - Malignant (1): akiec (0), bcc (1), mel (4)
    - Benign (0): bkl (2), df (3), nv (5), vasc (6)

    Args:
        labels: Original 7-class labels (0-6)

    Returns:
        Binary labels (0=Benign, 1=Malignant)
    """
    malignant_classes = [0, 1, 4]  # akiec, bcc, mel - CORRECTED!
    binary_labels = np.isin(labels.flatten(), malignant_classes).astype(int)
    return binary_labels

print("\n1️⃣ LOADING DATASETS...")
train_dataset = DermaMNIST(split='train', download=True, size=28)
val_dataset = DermaMNIST(split='val', download=True, size=28)
test_dataset = DermaMNIST(split='test', download=True, size=28)

# Extract and normalize images
print("2️⃣ NORMALIZING IMAGES (0-255 → 0-1)...")
x_train = train_dataset.imgs.astype(np.float32) / 255.0
x_val = val_dataset.imgs.astype(np.float32) / 255.0
x_test = test_dataset.imgs.astype(np.float32) / 255.0

# Transform labels to binary
print("3️⃣ TRANSFORMING LABELS TO BINARY...")
y_train_binary = transform_to_binary(train_dataset.labels)
y_val_binary = transform_to_binary(val_dataset.labels)
y_test_binary = transform_to_binary(test_dataset.labels)

# One-hot encode
y_train = tf.keras.utils.to_categorical(y_train_binary, num_classes=2)
y_val = tf.keras.utils.to_categorical(y_val_binary, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test_binary, num_classes=2)

# Verify binary distribution
print("\n4️⃣ BINARY CLASS DISTRIBUTION")
print("-"*40)

for name, labels in [('Train', y_train_binary), ('Val', y_val_binary), ('Test', y_test_binary)]:
    benign = np.sum(labels == 0)
    malignant = np.sum(labels == 1)
    total = len(labels)
    print(f"   {name:5}: Benign={benign:,} ({100*benign/total:.1f}%)  Malignant={malignant:,} ({100*malignant/total:.1f}%)")

# Calculate class imbalance ratio
train_benign = np.sum(y_train_binary == 0)
train_malignant = np.sum(y_train_binary == 1)
imbalance_ratio = train_benign / train_malignant

print(f"\n   Class Imbalance Ratio: {imbalance_ratio:.2f}:1 (Benign:Malignant)")

# Visualize binary distribution
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

for ax, (name, labels) in zip(axes, [('Train', y_train_binary), ('Val', y_val_binary), ('Test', y_test_binary)]):
    benign = np.sum(labels == 0)
    malignant = np.sum(labels == 1)

    bars = ax.bar(['Benign', 'Malignant'], [benign, malignant],
                  color=['forestgreen', 'crimson'], edgecolor='black')
    ax.set_title(f'{name} Set')
    ax.set_ylabel('Count')

    # Add count labels
    for bar, count in zip(bars, [benign, malignant]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20,
                f'{count:,}', ha='center', va='bottom', fontsize=10)

plt.suptitle('Binary Classification Distribution (CORRECTED)', fontsize=12)
plt.tight_layout()
plt.show()

data = {
    'x_train': x_train,
    'y_train': y_train,              # One-hot encoded [1,0] or [0,1]
    'y_train_bin': y_train_binary,   # Binary: 0 or 1
    'x_val': x_val,
    'y_val': y_val,                  # One-hot encoded
    'y_val_bin': y_val_binary,       # Binary: 0 or 1
    'x_test': x_test,
    'y_test': y_test,                # One-hot encoded
    'y_test_bin': y_test_binary,     # Binary: 0 or 1
    'class_weights': None            # Will be set in Block 6
}

print("\n5️⃣ DATA SHAPES")
print("-"*40)
print(f"   x_train: {x_train.shape}  y_train: {y_train.shape}  y_train_bin: {y_train_binary.shape}")
print(f"   x_val:   {x_val.shape}  y_val:   {y_val.shape}  y_val_bin:   {y_val_binary.shape}")
print(f"   x_test:  {x_test.shape}  y_test:  {y_test.shape}  y_test_bin:  {y_test_binary.shape}")

# Data augmentation
print("\n6️⃣ DATA AUGMENTATION")
print("-"*40)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

print("   ✅ ImageDataGenerator configured")
print("   Augmentations: rotation, shift, shear, zoom, flip")

print("\n" + "="*60)
print("✅ DATA TRANSFORMATION COMPLETE (CORRECTED LABELS)")
print("="*60)
print(f"""
   Binary Classes:
   ├── 0 = Benign    (bkl, df, nv, vasc)
   └── 1 = Malignant (akiec, bcc, mel)

   Class Imbalance: {imbalance_ratio:.1f}:1
   → Recommend using class weights (W3 or W5)

   Data Dictionary Keys:
   ├── x_train, x_val, x_test     (images)
   ├── y_train, y_val, y_test     (one-hot labels)
   └── y_train_bin, y_val_bin, y_test_bin (binary labels)
""")

In [ ]:
# ============================================================================
# BLOCK 2c: EDA - TRANSFORMED DATA & DEFINE TRAIN, VAL & TEST DATA
# ============================================================================

print("\n" + "="*60)
print("📊 EXPLORATORY DATA ANALYSIS - TRANSFORMED DATA")
print("="*60)

print("\n1️⃣ TRANSFORMED DATA OVERVIEW")
print("-"*40)
print(f"   {'Split':<12} {'Shape':<20} {'Dtype':<12} {'Pixel Range'}")
print(f"   {'-'*12} {'-'*20} {'-'*12} {'-'*15}")
print(f"   {'Train':<12} {str(data['x_train'].shape):<20} {str(data['x_train'].dtype):<12} [{data['x_train'].min():.2f}, {data['x_train'].max():.2f}]")
print(f"   {'Val':<12} {str(data['x_val'].shape):<20} {str(data['x_val'].dtype):<12} [{data['x_val'].min():.2f}, {data['x_val'].max():.2f}]")
print(f"   {'Test':<12} {str(data['x_test'].shape):<20} {str(data['x_test'].dtype):<12} [{data['x_test'].min():.2f}, {data['x_test'].max():.2f}]")

print("\n2️⃣ BINARY LABEL DISTRIBUTION")
print("-"*40)

for split_name, y_bin in [('Train', data['y_train_bin']),
                           ('Val', data['y_val_bin']),
                           ('Test', data['y_test_bin'])]:
    total = len(y_bin)
    malignant = y_bin.sum()
    benign = total - malignant
    print(f"   {split_name}:")
    print(f"      Benign (0):    {benign:>5,} ({100*benign/total:>5.1f}%)")
    print(f"      Malignant (1): {int(malignant):>5,} ({100*malignant/total:>5.1f}%)")
    print(f"      Ratio: {benign/malignant:.1f}:1")
    print()

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, (split_name, y_bin) in zip(axes, [('Train', data['y_train_bin']),
                                           ('Val', data['y_val_bin']),
                                           ('Test', data['y_test_bin'])]):
    total = len(y_bin)
    malignant = int(y_bin.sum())
    benign = total - malignant

    bars = ax.bar(['Benign\n(0)', 'Malignant\n(1)'], [benign, malignant],
                  color=['forestgreen', 'crimson'], edgecolor='black')
    ax.set_title(f'{split_name} Set (n={total:,})')
    ax.set_ylabel('Count')

    # Add labels
    for bar, count in zip(bars, [benign, malignant]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
                f'{count:,}\n({100*count/total:.1f}%)', ha='center', va='bottom', fontsize=10)

plt.suptitle('Binary Class Distribution After Transformation', fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

print("\n3️⃣ SAMPLE IMAGES BY BINARY CLASS")
print("-"*40)

fig, axes = plt.subplots(2, 6, figsize=(14, 5))

y_train_flat = data['y_train_bin'].flatten()

for row, (cls, cls_name, color) in enumerate([(0, 'Benign', 'forestgreen'),
                                                (1, 'Malignant', 'crimson')]):
    cls_indices = np.where(y_train_flat == cls)[0]

    for col in range(6):
        idx = cls_indices[col]
        axes[row, col].imshow(data['x_train'][idx])
        axes[row, col].axis('off')
        if col == 0:
            axes[row, col].set_ylabel(cls_name, fontsize=12, color=color, fontweight='bold')

axes[0, 0].set_title('Benign Samples (Class 0)', fontsize=10, loc='left', color='forestgreen')
axes[1, 0].set_title('Malignant Samples (Class 1)', fontsize=10, loc='left', color='crimson')

plt.suptitle('Sample Images After Binary Transformation', fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

print("\n4️⃣ ONE-HOT ENCODING VERIFICATION")
print("-"*40)
print(f"   y_train shape: {data['y_train'].shape}")
print(f"   y_train sample (first 5):")
print(f"   {data['y_train'][:5]}")

print("\n5️⃣ DATA AUGMENTATION PREVIEW")
print("-"*40)

# Pick one sample
sample_idx = np.where(data['y_train_bin'].flatten() == 1)[0][0]  # First malignant
sample_img = data['x_train'][sample_idx:sample_idx+1]

fig, axes = plt.subplots(2, 5, figsize=(12, 5))

# Original
axes[0, 0].imshow(sample_img[0])
axes[0, 0].set_title('Original', fontweight='bold')
axes[0, 0].axis('off')

# Augmented versions
aug_iter = datagen.flow(sample_img, batch_size=1, seed=SEED)
for i in range(1, 10):
    row = i // 5
    col = i % 5
    aug_img = next(aug_iter)[0]
    axes[row, col].imshow(aug_img)
    axes[row, col].set_title(f'Augmented {i}')
    axes[row, col].axis('off')

plt.suptitle('Data Augmentation Examples (Same Image)', fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

print("\n6️⃣ CLASS IMBALANCE SUMMARY")
print("-"*40)

# Calculate actual ratio from data
train_mal = data['y_train_bin'].sum()
train_ben = len(data['y_train_bin']) - train_mal
actual_ratio = train_ben / train_mal

print(f"   ⚠️  Your data is IMBALANCED")
print(f"   ")
print(f"   Benign:Malignant ratio = ~{actual_ratio:.0f}:1")
print(f"   ")
print(f"   Without intervention, model will likely:")
print(f"      • Predict 'Benign' most of the time ({100*train_ben/len(data['y_train_bin']):.0f}% accuracy!)")
print(f"      • Have LOW recall for Malignant class")
print(f"   ")
print(f"   Strategies to address:")
print(f"      • Class weights")

print("\n" + "="*60)
print("✅ EDA COMPLETE - Ready for modeling")
print("="*60)

# Block 3: Model Architectures

## 1. Custom CNN (`build_custom_cnn`)
Lightweight CNN optimized for native 28×28 resolution with VGG-style double convolutions.

| Parameter | Description |
|-----------|-------------|
| `filters_base` | Initial filters (doubles per block) |
| `depth` | Number of conv blocks (2-4) |
| `dropout` | Dropout rate |
| `dense_units` | Dense layer size |

## 2. Hybrid CNN with CBAM (`build_hybrid_cnn`)
CNN with Convolutional Block Attention Module for channel and spatial attention.

## 3. Transfer Learning (`build_transfer_model`)
Pre-trained models with custom classification head:
- **MobileNetV2** (3.4M params) - Lightweight, min input 32×32
- **EfficientNetB0** (5.3M params) - Compound scaling
- **DenseNet121** (8M params) - Dense connections, better gradient flow

All transfer models upsample 28×28 → 56×56 using bilinear interpolation.

In [ ]:
## ============================================================================
# BLOCK 3: MODEL ARCHITECTURES
# ============================================================================
#   1. Custom CNN        - Optimized baseline CNN (VGG-style double conv)
#   2. Hybrid CNN (CBAM) - CNN with Channel & Spatial Attention (like MedNet)
#   3. Transfer Learning - MobileNetV2 & EfficientNetB0 (lightweight, for small images)

def build_custom_cnn(filters_base=32, depth=3, dropout=0.5, dense_units=256):
    """
    Optimized CNN baseline for 28x28 images.

    Improvements over naive CNN:
    - Double Conv per block (VGG-style) for better feature extraction
    - GlobalAveragePooling instead of Flatten (fewer params, more robust)
    - Constant dropout (not increasing) to avoid over-regularization
    - Filter cap at 4x base to prevent parameter explosion

    Args:
        filters_base: Starting number of filters (32 or 64 recommended)
        depth: Number of Conv blocks (3-4 recommended for 28x28)
        dropout: Dropout rate (0.4-0.5 recommended)
        dense_units: Neurons in dense layer (128-256 recommended)

    Returns:
        Keras Sequential model
    """
    model = models.Sequential([layers.Input(shape=(28, 28, 3))])

    for i in range(depth):
        # Cap filters at 4x base (e.g., 32→64→128→128, not 32→64→128→256→512)
        filters = filters_base * (2 ** min(i, 2))

        # Double Conv per block (VGG-style) - better feature extraction
        model.add(layers.Conv2D(filters, (3, 3), padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.Activation('relu'))

        model.add(layers.Conv2D(filters, (3, 3), padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.Activation('relu'))

        model.add(layers.MaxPooling2D((2, 2)))

        # Constant dropout per block (not increasing!)
        model.add(layers.Dropout(dropout * 0.5))

    # GlobalAveragePooling instead of Flatten
    # - Fewer parameters
    # - More robust to spatial variations
    # - Works better with small images
    model.add(layers.GlobalAveragePooling2D())

    # Classification head
    model.add(layers.Dense(dense_units, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(2, activation='softmax'))

    return model

def channel_attention(input_tensor, ratio=8):
    """
    Channel Attention Module.
    Focuses on 'what' is meaningful in the feature maps.

    Args:
        input_tensor: Input feature map
        ratio: Reduction ratio for the bottleneck

    Returns:
        Attention-weighted feature map
    """
    channels = input_tensor.shape[-1]

    # Global Average Pooling
    avg_pool = layers.GlobalAveragePooling2D()(input_tensor)
    avg_pool = layers.Reshape((1, 1, channels))(avg_pool)

    # Global Max Pooling
    max_pool = layers.GlobalMaxPooling2D()(input_tensor)
    max_pool = layers.Reshape((1, 1, channels))(max_pool)

    # Shared MLP
    shared_dense1 = layers.Dense(channels // ratio, activation='relu')
    shared_dense2 = layers.Dense(channels, activation='linear')

    avg_out = shared_dense2(shared_dense1(avg_pool))
    max_out = shared_dense2(shared_dense1(max_pool))

    # Combine and apply sigmoid
    attention = layers.Add()([avg_out, max_out])
    attention = layers.Activation('sigmoid')(attention)

    return layers.Multiply()([input_tensor, attention])

def spatial_attention(input_tensor, kernel_size=7):
    """
    Spatial Attention Module.
    Focuses on 'where' is meaningful in the feature maps.

    Args:
        input_tensor: Input feature map
        kernel_size: Size of convolution kernel

    Returns:
        Attention-weighted feature map
    """
    # Average and Max along channel axis
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(input_tensor)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(input_tensor)

    # Concatenate
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])

    # Convolution
    attention = layers.Conv2D(1, kernel_size, padding='same', activation='sigmoid')(concat)

    return layers.Multiply()([input_tensor, attention])

def cbam_block(input_tensor, ratio=8, kernel_size=7):
    """
    CBAM: Convolutional Block Attention Module.
    Combines Channel and Spatial Attention sequentially.

    Reference: "CBAM: Convolutional Block Attention Module" (Woo et al., 2018)

    Args:
        input_tensor: Input feature map
        ratio: Channel attention reduction ratio
        kernel_size: Spatial attention kernel size

    Returns:
        Attention-refined feature map
    """
    x = channel_attention(input_tensor, ratio)
    x = spatial_attention(x, kernel_size)
    return x

def build_hybrid_cnn(filters_base=64, depth=3, dropout=0.5, dense_units=256, use_cbam=True):
    """
    Hybrid CNN with CBAM Attention - Similar to MedNet architecture.

    This combines:
    - Standard Convolutions (more stable than SeparableConv for small images)
    - CBAM Attention (focuses on relevant features)
    - Global Average Pooling (more robust than Flatten)

    Inspired by MedNet which achieved 0.840 accuracy on DermaMNIST.

    Args:
        filters_base: Starting filters (64 recommended)
        depth: Number of blocks (3 recommended for 28x28)
        dropout: Dropout rate (0.5 recommended)
        dense_units: Dense layer neurons (256 recommended)
        use_cbam: Whether to use CBAM attention

    Returns:
        Keras Functional model
    """
    inputs = layers.Input(shape=(28, 28, 3))
    x = inputs

    # Initial convolution
    x = layers.Conv2D(filters_base, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    for i in range(depth):
        # Cap filters at 4x base (256 max for filters_base=64)
        filters = filters_base * (2 ** min(i, 2))

        # Double Conv per block (VGG-style) - more stable than SeparableConv
        x = layers.Conv2D(filters, (3, 3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)

        x = layers.Conv2D(filters, (3, 3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)

        # CBAM Attention after each block (only if feature map large enough)
        if use_cbam and x.shape[1] >= 7:  # CBAM needs at least 7x7 for kernel_size=7
            x = cbam_block(x, ratio=8, kernel_size=3)  # Smaller kernel for small feature maps

        # MaxPooling
        x = layers.MaxPooling2D((2, 2))(x)

        # Constant dropout (not increasing)
        x = layers.Dropout(dropout * 0.5)(x)

    # Global pooling instead of flatten (more robust, fewer parameters)
    x = layers.GlobalAveragePooling2D()(x)

    # Classification head
    x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(2, activation='softmax')(x)

    model_name = "hybrid_cnn_cbam" if use_cbam else "hybrid_cnn"
    return models.Model(inputs, outputs, name=model_name)

def build_transfer_model(base_name='mobilenetv2', dropout=0.5, unfreeze_layers=None, dense_units=256):
    """
    Transfer Learning with lightweight models optimized for small images.

    Available Models:
    - MobileNetV2:    3.4M params, min input 32x32, designed for mobile/edge
    - EfficientNetB0: 5.3M params, efficient compound scaling
    - DenseNet121:    8M params, dense connections (better for small feature maps)

    Args:
        base_name: 'mobilenetv2', 'efficientnet', or 'densenet'
        dropout: Dropout rate
        unfreeze_layers: Fine-tuning strategy
            - None: Full fine-tuning (all layers trainable)
            - 0: Feature extraction only (all layers frozen)
            - N: Unfreeze last N layers
        dense_units: Dense layer neurons

    Returns:
        Keras Functional model
    """
    inputs = layers.Input(shape=(28, 28, 3))

    # Upsampling: 28x28 → 56x56 (minimum for decent feature extraction)
    # Note: These models work best at 224x224, but 56x56 is a compromise
    x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(inputs)

    # Available base models (lightweight, suitable for small images)
    base_models = {
        'mobilenetv2': lambda: MobileNetV2(
            include_top=False,
            weights='imagenet',
            input_shape=(56, 56, 3),
            pooling='avg'
        ),
        'efficientnet': lambda: EfficientNetB0(
            include_top=False,
            weights='imagenet',
            input_shape=(56, 56, 3),
            pooling='avg'
        ),
        'densenet': lambda: DenseNet121(
            include_top=False,
            weights='imagenet',
            input_shape=(56, 56, 3),
            pooling='avg'
        )
    }

    if base_name not in base_models:
        raise ValueError(f"Unknown base: {base_name}. Choose from: {list(base_models.keys())}")

    base = base_models[base_name]()

    print(f"   {base_name}: {len(base.layers)} total layers")

    # Fine-tuning strategy
    if unfreeze_layers is None:
        # Full fine-tuning - all layers trainable
        base.trainable = True
        print(f"   Mode: Full fine-tuning (all {len(base.layers)} layers trainable)")
    elif unfreeze_layers == 0:
        # Feature extraction - all layers frozen
        base.trainable = False
        print(f"   Mode: Feature extraction (all layers frozen)")
    else:
        # Partial fine-tuning - freeze early layers, train later ones
        base.trainable = True
        for layer in base.layers[:-unfreeze_layers]:
            layer.trainable = False
        trainable_count = sum([1 for l in base.layers if l.trainable])
        print(f"   Mode: Partial fine-tuning ({trainable_count}/{len(base.layers)} layers trainable)")

    # Forward pass through base model (training=False for BatchNorm layers)
    x = base(x, training=False)

    # Classification head
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout * 0.5)(x)
    outputs = layers.Dense(2, activation='softmax')(x)

    return models.Model(inputs, outputs, name=f"{base_name}_model")

# SUMMARY

print("=" * 70)
print("✅ BLOCK 3: MODEL ARCHITECTURES LOADED")
print("=" * 70)
print("""

   Key Improvements in Custom CNN:
   ├── Double Conv per block (VGG-style)
   ├── GlobalAveragePooling (not Flatten)
   ├── Constant dropout (not increasing)
   └── Filter cap at 4x base

   DenseNet121 Advantages:
   ├── Dense connections → better gradient flow
   ├── Feature reuse → efficient for small images
   └── Min input 32×32 → works well with 56×56 upsampling
""")

# Block 4: Training Configuration & Helpers

## Key Components

### `compile_model(model, learning_rate)`
Compiles with Adam optimizer, categorical crossentropy, and clinical metrics.

### `get_callbacks(config, model_path)`
Creates callbacks with **F2 Score** monitoring (not validation loss).

**Why F2 Score?**
$$F_2 = \frac{5 \times \text{Precision} \times \text{Recall}}{4 \times \text{Precision} + \text{Recall}}$$

- Weights Recall 2× more than Precision
- Prevents "all Malignant" predictions (would have low precision → low F2)
- Prevents "all Benign" predictions (would have zero recall → F2 = 0)
- Aligns directly with clinical priority: **don't miss cancer!**

| Callback | Monitor | Mode | Purpose |
|----------|---------|------|---------|
| EarlyStopping | val_f2 | max | Stop when F2 plateaus |
| ModelCheckpoint | val_f2 | max | Save best F2 model |

In [ ]:
# ============================================================================
# BLOCK 4: TRAINING CONFIGURATION & HELPERS (WITH F2 SCORE)
# ============================================================================

MONITOR_METRIC = 'val_f2_score'
MONITOR_MODE = 'max'

# F2 SCORE METRIC

class F2Score(tf.keras.metrics.Metric):
    """
    F2 Score metric that weights Recall 2x more than Precision.

    F2 = 5 × (Precision × Recall) / (4 × Precision + Recall)

    For binary classification with class_id=1 (Malignant).
    """

    def __init__(self, name='f2_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_classes = tf.argmax(y_pred, axis=1)
        y_true_classes = tf.argmax(y_true, axis=1)

        # Cast to float for calculations
        y_pred_classes = tf.cast(y_pred_classes, tf.float32)
        y_true_classes = tf.cast(y_true_classes, tf.float32)

        # Calculate TP, FP, FN for class 1 (Malignant)
        tp = tf.reduce_sum(y_true_classes * y_pred_classes)
        fp = tf.reduce_sum((1 - y_true_classes) * y_pred_classes)
        fn = tf.reduce_sum(y_true_classes * (1 - y_pred_classes))

        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())

        # F2 formula: (1 + beta^2) * (precision * recall) / (beta^2 * precision + recall)
        # beta = 2 for F2
        f2 = 5 * (precision * recall) / (4 * precision + recall + tf.keras.backend.epsilon())

        return f2

    def reset_state(self):
        self.true_positives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)

# W&B METRICS LOGGER (with Loss)

class WandbGroupedMetricsLogger(tf.keras.callbacks.Callback):
    """
    Logs metrics in grouped format for better W&B visualization.

    Groups:
    - loss/train, loss/val
    - f2/train, f2/val
    - recall/train, recall/val
    - precision/train, precision/val
    - auc/train, auc/val
    """

    def on_epoch_end(self, epoch, logs=None):
        if not USE_WANDB or wandb is None:
            return

        logs = logs or {}

        grouped_logs = {
            'epoch': epoch,
            # Loss
            'loss/train': logs.get('loss'),
            'loss/val': logs.get('val_loss'),
            # F2 Score (Early Stopping Metric)
            'f2/train': logs.get('f2_score'),
            'f2/val': logs.get('val_f2_score'),
            # Recall (primary clinical metric)
            'recall/train': logs.get('recall'),
            'recall/val': logs.get('val_recall'),
            # Precision
            'precision/train': logs.get('precision'),
            'precision/val': logs.get('val_precision'),
            # AUC
            'auc/train': logs.get('auc'),
            'auc/val': logs.get('val_auc'),
        }

        # Remove None values
        grouped_logs = {k: v for k, v in grouped_logs.items() if v is not None}

        wandb.log(grouped_logs)

# MODEL COMPILATION

def compile_model(model, learning_rate):
    """
    Compile model with all metrics including F2 Score.

    Metrics:
    - loss (categorical_crossentropy)
    - auc
    - precision (for Malignant class)
    - recall (for Malignant class)
    - f2_score (custom)
    """
    model.compile(
        optimizer=optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=[
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.Precision(class_id=1, name='precision'),
            tf.keras.metrics.Recall(class_id=1, name='recall'),
            F2Score(name='f2_score')
        ]
    )

# SUMMARY

print("=" * 60)
print("✅ BLOCK 4: TRAINING CONFIGURATION LOADED")
print("=" * 60)
print(f"""
   Monitoring Metric: {MONITOR_METRIC} ({MONITOR_MODE})

   F2 Score Formula:
   F2 = 5 × (Precision × Recall) / (4 × Precision + Recall)

   Why F2?
   ├── Weights Recall 2x more than Precision
   ├── Ideal for medical diagnosis (don't miss cancer!)
   ├── Prevents "all Malignant" models (Precision too low)
   └── Prevents "all Benign" models (Recall too low)

   Metrics logged to W&B:
   ├── loss/train, loss/val
   ├── f2/train, f2/val
   ├── recall/train, recall/val
   ├── precision/train, precision/val
   └── auc/train, auc/val
""")

# Block 5: Main Training Loop

## `train_experiment(config, data, datagen)`

Orchestrates the complete training pipeline for any architecture.

### Pipeline Steps
1. **Initialize W&B** — Start experiment tracking
2. **Build Model** — Custom CNN, Hybrid CNN, or Transfer Learning
3. **Create Generator** — Infinite augmented data stream
4. **Train** — Fit with callbacks, class weights, and F2 monitoring
5. **Find Best Epoch** — Select epoch with highest validation F2
6. **Confusion Matrix** — Compute TP/FP/TN/FN at best epoch
7. **Log Results** — Save all metrics to W&B

### Returns
Dictionary with best epoch metrics: F2, Recall, Precision, AUC, Confusion Matrix

In [ ]:
# ============================================================================
# BLOCK 5: TRAINING LOOP WITH F2-BASED EARLY STOPPING
# ============================================================================
# This block contains the main training function that:
#   - Builds the appropriate model based on config['architecture']
#   - Trains with F2-Score monitoring
#   - Logs to W&B
#   - Returns results dictionary

def train_experiment(config, data, datagen):
    """
    Train a single experiment with the given configuration.

    Supports architectures:
        - 'custom_cnn': build_custom_cnn()
        - 'hybrid_cnn': build_hybrid_cnn()
        - 'transfer':   build_transfer_model()

    Args:
        config: Dictionary with model configuration
        data: Dictionary with train/val/test data and class weights
        datagen: ImageDataGenerator for augmentation

    Returns:
        Dictionary with training results
    """

    print("\n" + "=" * 60)
    print(f"🚀 STARTING: {config['name']}")
    print("=" * 60)

    # ────────────────────────────────────────────────────────────────
    # 1. BUILD MODEL based on architecture type
    # ────────────────────────────────────────────────────────────────
    arch = config['architecture']

    if arch == 'custom_cnn':
        model = build_custom_cnn(
            filters_base=config.get('filters_base', 32),
            depth=config.get('depth', 3),
            dropout=config.get('dropout', 0.5),
            dense_units=config.get('dense_units', 512)
        )

    elif arch == 'hybrid_cnn':
        model = build_hybrid_cnn(
            filters_base=config.get('filters_base', 64),
            depth=config.get('depth', 4),
            dropout=config.get('dropout', 0.5),
            dense_units=config.get('dense_units', 256),
            use_cbam=config.get('use_cbam', True)
        )

    elif arch == 'transfer':
        model = build_transfer_model(
            base_name=config.get('base_name', 'mobilenetv2'),
            dropout=config.get('dropout', 0.5),
            unfreeze_layers=config.get('unfreeze_layers', None),
            dense_units=config.get('dense_units', 256)
        )

    else:
        raise ValueError(f"Unknown architecture: {arch}")

    # ────────────────────────────────────────────────────────────────
    # 2. COMPILE MODEL with F2 metric
    # ────────────────────────────────────────────────────────────────
    # Note: compile_model() modifies model in-place, doesn't return anything
    compile_model(model, learning_rate=config.get('learning_rate', 0.001))

    # ────────────────────────────────────────────────────────────────
    # 3. SETUP CALLBACKS
    # ────────────────────────────────────────────────────────────────
    callback_list = [
        callbacks.EarlyStopping(
            monitor=MONITOR_METRIC,
            patience=config.get('patience', 10),
            mode=MONITOR_MODE,
            restore_best_weights=True,
            verbose=1
        ),
        callbacks.ModelCheckpoint(
            filepath=f"{config['name']}.keras",
            monitor=MONITOR_METRIC,
            mode=MONITOR_MODE,
            save_best_only=True,
            verbose=1
        )
    ]

    # Add W&B callback if enabled
    if USE_WANDB and wandb is not None:
        run = wandb.init(
            project=PROJECT_NAME,
            name=config['name'],
            config=config,
            reinit=True
        )
        callback_list.append(WandbGroupedMetricsLogger())

    # ────────────────────────────────────────────────────────────────
    # 4. PREPARE DATA
    # ────────────────────────────────────────────────────────────────
    batch_size = config.get('batch_size', 64)

    train_generator = datagen.flow(
        data['x_train'], data['y_train'],
        batch_size=batch_size,
        shuffle=True
    )

    # Validation data (no augmentation)
    val_data = (data['x_val'], data['y_val'])

    # ────────────────────────────────────────────────────────────────
    # 5. TRAIN MODEL
    # ────────────────────────────────────────────────────────────────
    history = model.fit(
        train_generator,
        epochs=config.get('epochs', 50),
        validation_data=val_data,
        class_weight=data['class_weights'],
        callbacks=callback_list,
        verbose=1
    )

    # ────────────────────────────────────────────────────────────────
    # 6. EVALUATE BEST MODEL
    # ────────────────────────────────────────────────────────────────
    print("\n📊 Generating Confusion Matrix for BEST Model...")

    # Find best epoch by F2
    val_f2_history = history.history['val_f2_score']
    best_epoch_idx = np.argmax(val_f2_history)

    # Get metrics at best epoch
    best_val_f2 = val_f2_history[best_epoch_idx]
    best_val_recall = history.history['val_recall'][best_epoch_idx]
    best_val_precision = history.history['val_precision'][best_epoch_idx]
    best_val_auc = history.history['val_auc'][best_epoch_idx]

    # Generate predictions on validation set
    y_pred_proba = model.predict(data['x_val'], verbose=0)
    y_pred = np.argmax(y_pred_proba, axis=1)
    y_true = np.argmax(data['y_val'], axis=1)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    # Calculate F2 from confusion matrix for verification
    precision_cm = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall_cm = tp / (tp + fn) if (tp + fn) > 0 else 0
    f2_cm = 5 * (precision_cm * recall_cm) / (4 * precision_cm + recall_cm) if (4 * precision_cm + recall_cm) > 0 else 0

    print(f"""
   Confusion Matrix:
                    Predicted
                  Benign  Malignant
   Actual Benign   (TN) {tn:4d}   (FP) {fp:4d}
          Malig.   (FN) {fn:4d}   (TP) {tp:4d}

   ❌ Missed Cancers (FN): {fn}
   ⚠️  False Alarms (FP):  {fp}""")

    # ────────────────────────────────────────────────────────────────
    # 7. SAVE CONFUSION MATRIX PLOT
    # ────────────────────────────────────────────────────────────────
    save_path = f"cm_{config['name']}.png"
    plot_confusion_matrix(cm, config['name'], save_path,
                          recall=best_val_recall,
                          precision=best_val_precision,
                          f2=best_val_f2)
    print(f"📊 Confusion matrix saved: {save_path}")

    # ────────────────────────────────────────────────────────────────
    # 8. LOG BEST MODEL METRICS TO W&B SUMMARY
    # ────────────────────────────────────────────────────────────────
    if USE_WANDB and wandb is not None:
        # Use wandb.summary for final/best metrics (not wandb.log!)
        # These appear in the W&B Table for easy model comparison
        wandb.summary['best_val_f2'] = best_val_f2
        wandb.summary['best_val_recall'] = best_val_recall
        wandb.summary['best_val_precision'] = best_val_precision
        wandb.summary['best_val_auc'] = best_val_auc
        wandb.summary['best_val_loss'] = history.history['val_loss'][best_epoch_idx]
        wandb.summary['best_epoch'] = best_epoch_idx + 1
        wandb.summary['total_epochs'] = len(history.history['loss'])

        # Confusion matrix values
        wandb.summary['cm_tn'] = tn
        wandb.summary['cm_fp'] = fp
        wandb.summary['cm_fn'] = fn
        wandb.summary['cm_tp'] = tp

        # ────────────────────────────────────────────────────────────
        # VISUALIZE BEST EPOCH IN TRAINING CURVES
        # ────────────────────────────────────────────────────────────
        # Create a custom chart showing best epoch marker
        epochs_range = list(range(1, len(history.history['val_f2_score']) + 1))

        # Create training curves plot with best epoch highlighted
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))

        # F2 Score
        axes[0, 0].plot(epochs_range, history.history['f2_score'], 'b-', label='Train F2')
        axes[0, 0].plot(epochs_range, history.history['val_f2_score'], 'r-', label='Val F2')
        axes[0, 0].axvline(x=best_epoch_idx + 1, color='g', linestyle='--', label=f'Best Epoch ({best_epoch_idx + 1})')
        axes[0, 0].scatter([best_epoch_idx + 1], [best_val_f2], color='g', s=100, zorder=5)
        axes[0, 0].set_title(f'F2 Score (Best: {best_val_f2:.4f})')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

        # Loss
        axes[0, 1].plot(epochs_range, history.history['loss'], 'b-', label='Train Loss')
        axes[0, 1].plot(epochs_range, history.history['val_loss'], 'r-', label='Val Loss')
        axes[0, 1].axvline(x=best_epoch_idx + 1, color='g', linestyle='--', label=f'Best Epoch ({best_epoch_idx + 1})')
        axes[0, 1].set_title('Loss')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

        # Recall
        axes[1, 0].plot(epochs_range, history.history['recall'], 'b-', label='Train Recall')
        axes[1, 0].plot(epochs_range, history.history['val_recall'], 'r-', label='Val Recall')
        axes[1, 0].axvline(x=best_epoch_idx + 1, color='g', linestyle='--', label=f'Best Epoch ({best_epoch_idx + 1})')
        axes[1, 0].scatter([best_epoch_idx + 1], [best_val_recall], color='g', s=100, zorder=5)
        axes[1, 0].set_title(f'Recall (Best: {best_val_recall:.4f})')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)

        # Precision
        axes[1, 1].plot(epochs_range, history.history['precision'], 'b-', label='Train Precision')
        axes[1, 1].plot(epochs_range, history.history['val_precision'], 'r-', label='Val Precision')
        axes[1, 1].axvline(x=best_epoch_idx + 1, color='g', linestyle='--', label=f'Best Epoch ({best_epoch_idx + 1})')
        axes[1, 1].scatter([best_epoch_idx + 1], [best_val_precision], color='g', s=100, zorder=5)
        axes[1, 1].set_title(f'Precision (Best: {best_val_precision:.4f})')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)

        plt.suptitle(f"{config['name']} - Training Curves (Best Epoch: {best_epoch_idx + 1})", fontsize=14)
        plt.tight_layout()

        # Save and log to W&B
        curves_path = f"curves_{config['name']}.png"
        plt.savefig(curves_path, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"📈 Training curves saved: {curves_path}")

        wandb.log({'training_curves': wandb.Image(curves_path)})

        # Log confusion matrix as image
        wandb.log({'confusion_matrix': wandb.Image(save_path)})

        # Log interactive confusion matrix
        wandb.log({
            'conf_mat_interactive': wandb.plot.confusion_matrix(
                y_true=y_true,
                preds=y_pred,
                class_names=['Benign', 'Malignant']
            )
        })

        wandb.finish()

    # ────────────────────────────────────────────────────────────────
    # 9. PREPARE RESULTS DICTIONARY
    # ────────────────────────────────────────────────────────────────
    results = {
        'name': config['name'],
        'architecture': arch,
        'best_epoch': best_epoch_idx + 1,
        'best_val_f2': best_val_f2,
        'best_val_recall': best_val_recall,
        'best_val_precision': best_val_precision,
        'best_val_auc': best_val_auc,
        'f2_from_cm': f2_cm,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tp': tp,
        'config': config
    }

    print(f"\n✅ Results Logged. Best Epoch: {best_epoch_idx + 1} (by val_f2_score)")
    print(f"   Val F2:        {best_val_f2:.4f}  ← Early Stopping Metric")
    print(f"   Val Recall:    {best_val_recall:.4f}")
    print(f"   Val Precision: {best_val_precision:.4f}")
    print(f"   Val AUC:       {best_val_auc:.4f}")

    # ────────────────────────────────────────────────────────────────
    # 10. CLEANUP
    # ────────────────────────────────────────────────────────────────
    K.clear_session()
    gc.collect()

    return results

# ════════════════════════════════════════════════════════════════════════════
# HELPER: CONFUSION MATRIX PLOT
# ════════════════════════════════════════════════════════════════════════════

def plot_confusion_matrix(cm, title, save_path, recall=None, precision=None, f2=None):
    """
    Plot and save a confusion matrix with metrics.

    Args:
        cm: 2x2 confusion matrix
        title: Plot title
        save_path: Path to save the figure
        recall, precision, f2: Optional metrics to display
    """
    plt.figure(figsize=(8, 6))

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Benign', 'Malignant'],
                yticklabels=['Benign', 'Malignant'],
                annot_kws={'size': 16})

    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('Actual', fontsize=12)

    title_text = f'{title}'
    if f2 is not None:
        title_text += f'\nF2={f2:.3f}'
    if recall is not None:
        title_text += f' | Recall={recall:.3f}'
    if precision is not None:
        title_text += f' | Precision={precision:.3f}'

    plt.title(title_text, fontsize=12)
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()

print("=" * 70)
print("✅ BLOCK 5: TRAINING LOOP LOADED")
print("=" * 70)
print("""
   train_experiment(config, data, datagen)

   Supported architectures:
   ├── 'custom_cnn'  → build_custom_cnn()
   ├── 'hybrid_cnn'  → build_hybrid_cnn()
   └── 'transfer'    → build_transfer_model()

   Features:
   ├── F2-Score Early Stopping
   ├── W&B Logging (if enabled)
   ├── Confusion Matrix Generation
   └── Automatic Cleanup
""")

# Block 6: Phase 1 — Architecture Screening

## Goal
Find the best **architecture + fine-tuning strategy** combination for malignant lesion detection.

## Experimental Design (14 runs)

| Architecture   | Variants                                                 | Runs | Parameters |
|----------------|----------------------------------------------------------|------|------------|
| Custom CNN     | shallow (d=3, f=32), deep (d=4, f=64), wide (d=3, f=128) | 3    | ~50K-200K  |
| Hybrid CNN     | with CBAM, without CBAM                                  | 2    | ~80K-100K  |
| MobileNetV2    | frozen, partial (50 layers), full                        | 3    | 3.4M       |
| EfficientNetB0 | frozen, partial (100 layers), full                       | 3    | 5.3M       |
| DenseNet121    | frozen, partial (50 layers), full                        | 3    | 8M         |
| **Total**      |                                                          | **14** |          |

## Fixed Parameters

All experiments use **W3** class weighting: `{0: 1.0, 1: 3.0}` — penalizes missed malignant cases 3× more than false alarms.

## Selection Criterion

Best model selected by **validation F2 score** (weights recall 2× over precision).

In [ ]:
#============================================================================
# BLOCK 6: Phase 1 — Architecture Screening
# ============================================================================

WEIGHT_TAG = "W3"
data['class_weights'] = {0: 1.0, 1: 3.0}

print(f"✅ Class weights set: {data['class_weights']}")

phase1_results = []

# Block 6a: Custom CNN (3 runs)

Tests three CNN configurations varying depth and width to find optimal complexity for 28×28 images.

| Name | Depth | Filters | Hypothesis |
|------|-------|---------|------------|
| P1_CNN_shallow | 3 | 32 | Simple may be sufficient |
| P1_CNN_deep | 4 | 64 | More depth = more features |
| P1_CNN_wide | 3 | 128 | More filters = more capacity |

**Why Custom CNN?**
- Native 28×28 input (no upsampling artifacts like transfer learning)
- Appropriate complexity for ~7K training samples (avoids overfitting)
- Fast training and inference

In [ ]:
# ============================================================================
# 6a: CUSTOM CNN (3 runs)
# ============================================================================

print(f"✅ Running Custom CNN experiments (3 runs)")

configs_cnn = [
    {
        # Shallow: Tests if simple architecture suffices for low-resolution images
        'name': f'P1_CNN_shallow_{WEIGHT_TAG}',
        'architecture': 'custom_cnn',
        'filters_base': 32,
        'depth': 3,
        'dropout': 0.4,
        'dense_units': 128,
        'learning_rate': 0.0005,
        'batch_size': 128,
        'epochs': 50,
        'patience': 10,
    },
    {
        # Deep: Tests if additional layers capture more discriminative features
        'name': f'P1_CNN_deep_{WEIGHT_TAG}',
        'architecture': 'custom_cnn',
        'filters_base': 64,
        'depth': 4,
        'dropout': 0.4,
        'dense_units': 256,
        'learning_rate': 0.0005,
        'batch_size': 128,
        'epochs': 50,
        'patience': 10,
    },
    {
        # Wide: Tests if more filters per layer improve feature extraction
        'name': f'P1_CNN_wide_{WEIGHT_TAG}',
        'architecture': 'custom_cnn',
        'filters_base': 128,
        'depth': 3,
        'dropout': 0.5,  # Higher dropout to compensate for increased capacity
        'dense_units': 256,
        'learning_rate': 0.0005,
        'batch_size': 128,
        'epochs': 50,
        'patience': 10,
    },
]

for i, config in enumerate(configs_cnn, 1):
    result = train_experiment(config, data, datagen)
    phase1_results.append(result)
    save_to_drive(config['name'], DIR_PHASE_1)

# Block 6b: Hybrid CNN with CBAM (2 runs)

Tests whether attention mechanisms improve feature extraction, inspired by MedNet (state-of-the-art on DermaMNIST).

| Name | CBAM | Purpose |
|------|------|---------|
| P1_Hybrid_CBAM | Yes | Full attention mechanism |
| P1_Hybrid_NoCBAM | No | Ablation study |

**CBAM Architecture:**
```
Channel Attention → "What" features are important?
       ↓
Spatial Attention → "Where" to focus?
```

**Why CBAM?**
- Proven effective for medical imaging (MedNet: 0.840 acc on DermaMNIST)
- Lightweight addition (~1% extra parameters)
- Ablation study validates attention contribution

In [ ]:
# ============================================================================
# 6b: HYBRID CNN WITH CBAM (2 runs)
# ============================================================================

print(f"✅ Running Hybrid CNN experiments (2 runs)")

configs_hybrid = [
    {
        # With CBAM: Tests if attention mechanism improves feature focus
        'name': f'P1_Hybrid_CBAM_{WEIGHT_TAG}',
        'architecture': 'hybrid_cnn',
        'filters_base': 64,
        'depth': 3,
        'dropout': 0.5,
        'dense_units': 256,
        'use_cbam': True,
        'learning_rate': 0.0005,
        'batch_size': 128,
        'epochs': 50,
        'patience': 10,
    },
    {
        # Without CBAM: Ablation study to isolate attention contribution
        'name': f'P1_Hybrid_NoCBAM_{WEIGHT_TAG}',
        'architecture': 'hybrid_cnn',
        'filters_base': 64,
        'depth': 3,
        'dropout': 0.5,
        'dense_units': 256,
        'use_cbam': False,
        'learning_rate': 0.0005,
        'batch_size': 128,
        'epochs': 50,
        'patience': 10,
    },
]

for i, config in enumerate(configs_hybrid, 1):
    result = train_experiment(config, data, datagen)
    phase1_results.append(result)
    save_to_drive(config['name'], DIR_PHASE_1)

# Block 6c: MobileNetV2 Transfer Learning (3 runs)

Tests transfer learning with a lightweight architecture designed for mobile/edge deployment.

| Name | Unfrozen Layers | Purpose |
|------|-----------------|---------|
| P1_MobileNetV2_frozen | 0 | Feature extraction only |
| P1_MobileNetV2_partial | 50 | Balance: preserve ImageNet features + adapt |
| P1_MobileNetV2_full | All | Full fine-tuning (risk: catastrophic forgetting) |

**Why MobileNetV2?**
```
├── Parameters: 3.4M (smallest transfer model)
├── Inverted residuals + depthwise separable convolutions
└── Min input 32×32 (works well at 56×56)
```

In [ ]:
# ============================================================================
# 6c: MOBILENETV2 TRANSFER LEARNING (3 runs)
# ============================================================================

print(f"✅ Running MobileNetV2 experiments (3 runs)")

configs_mobilenet = [
    {
        # Frozen: Uses ImageNet features as fixed feature extractor
        'name': f'P1_MobileNetV2_frozen_{WEIGHT_TAG}',
        'architecture': 'transfer',
        'base_name': 'mobilenetv2',
        'dropout': 0.5,
        'dense_units': 256,
        'unfreeze_layers': 0,
        'learning_rate': 0.0005,
        'batch_size': 64,
        'epochs': 30,
        'patience': 10,
    },
    {
        # Partial: Adapts top layers while preserving low-level features
        'name': f'P1_MobileNetV2_partial_{WEIGHT_TAG}',
        'architecture': 'transfer',
        'base_name': 'mobilenetv2',
        'dropout': 0.5,
        'dense_units': 256,
        'unfreeze_layers': 50,
        'learning_rate': 0.0001,
        'batch_size': 64,
        'epochs': 30,
        'patience': 10,
    },
    {
        # Full: Complete fine-tuning, lower LR to prevent catastrophic forgetting
        'name': f'P1_MobileNetV2_full_{WEIGHT_TAG}',
        'architecture': 'transfer',
        'base_name': 'mobilenetv2',
        'dropout': 0.5,
        'dense_units': 256,
        'unfreeze_layers': None,
        'learning_rate': 0.0001,
        'batch_size': 64,
        'epochs': 30,
        'patience': 10,
    },
]

for i, config in enumerate(configs_mobilenet, 1):
    result = train_experiment(config, data, datagen)
    phase1_results.append(result)
    save_to_drive(config['name'], DIR_PHASE_1)

# Block 6d: EfficientNetB0 Transfer Learning (3 runs)

Tests compound-scaled architecture using Neural Architecture Search (NAS).

| Name | Unfrozen Layers | Purpose |
|------|-----------------|---------|
| P1_EfficientNet_frozen | 0 | Feature extraction only |
| P1_EfficientNet_partial | 100 | Partial adaptation |
| P1_EfficientNet_full | All | Full fine-tuning |

**Why EfficientNetB0?**
```
├── Parameters: 5.3M (efficient scaling)
├── Compound scaling (depth, width, resolution)
└── State-of-the-art efficiency
```

⚠️ **Caveat:** Designed for 224×224 input — may underperform at 56×56.

In [ ]:
# ============================================================================
# 6d: EFFICIENTNETB0 TRANSFER LEARNING (3 runs)
print("\n" + "=" * 70)
print("📦 6d: EFFICIENTNETB0 TRANSFER LEARNING (3 runs)")
print("=" * 70)

print(f"""
   EfficientNetB0 Architecture:
   ├── Parameters: 5.3M (efficient scaling)
   ├── Uses compound scaling (depth, width, resolution)
   └── State-of-the-art efficiency

   Fine-tuning strategies:
   ├── frozen:  All layers frozen (feature extraction only)
   ├── partial: Last 100 layers trainable
   └── full:    All layers trainable

   Note: Lower LR (0.0001) and smaller batch (64) for pre-trained weights
""")

configs_efficientnet = [
    {
        'name': f'P1_EfficientNet_frozen_{WEIGHT_TAG}',
        'architecture': 'transfer',
        'base_name': 'efficientnet',
        'description': 'EfficientNetB0 Frozen (feature extraction)',
        'dropout': 0.5,
        'dense_units': 256,
        'unfreeze_layers': 0,  # All frozen
        'learning_rate': 0.0005,  # Higher LR ok for frozen base
        'batch_size': 64,
        'epochs': 30,
        'patience': 10,
    },
    {
        'name': f'P1_EfficientNet_partial_{WEIGHT_TAG}',
        'architecture': 'transfer',
        'base_name': 'efficientnet',
        'description': 'EfficientNetB0 Partial (100 layers unfrozen)',
        'dropout': 0.5,
        'dense_units': 256,
        'unfreeze_layers': 100,  # Last 100 layers trainable
        'learning_rate': 0.0001,  # Lower LR for fine-tuning
        'batch_size': 64,
        'epochs': 30,
        'patience': 10,
    },
    {
        'name': f'P1_EfficientNet_full_{WEIGHT_TAG}',
        'architecture': 'transfer',
        'base_name': 'efficientnet',
        'description': 'EfficientNetB0 Full (all layers unfrozen)',
        'dropout': 0.5,
        'dense_units': 256,
        'unfreeze_layers': None,  # All trainable
        'learning_rate': 0.0001,  # Lower LR for fine-tuning
        'batch_size': 64,
        'epochs': 30,
        'patience': 10,
    },
]

for i, config in enumerate(configs_efficientnet, 1):
    print(f"\n▶️ EfficientNetB0 [{i}/3] {config['description']}")
    result = train_experiment(config, data, datagen)
    phase1_results.append(result)
    save_to_drive(config['name'], DIR_PHASE_1)

# Block 6e: DenseNet121 Transfer Learning (3 runs)

Tests dense connectivity architecture with feature reuse across layers.

| Name | Unfrozen Layers | Purpose |
|------|-----------------|---------|
| P1_DenseNet_frozen | 0 | Feature extraction only |
| P1_DenseNet_partial | 50 | Partial adaptation |
| P1_DenseNet_full | All | Full fine-tuning (stable due to dense connections) |

**Why DenseNet121?**
```
├── Parameters: 8M (largest, but efficient feature reuse)
├── Dense connections prevent information loss at low resolution
├── Better gradient flow enables stable full fine-tuning
└── Min input 32×32 (works well at 56×56)
```

In [ ]:
# ============================================================================
# 6e: DENSENET121 TRANSFER LEARNING (3 runs)
# ============================================================================

print(f"✅ Running DenseNet121 experiments (3 runs)")

configs_densenet = [
    {
        # Frozen: Tests dense features as fixed extractor
        'name': f'P1_DenseNet_frozen_{WEIGHT_TAG}',
        'architecture': 'transfer',
        'base_name': 'densenet',
        'dropout': 0.5,
        'dense_units': 256,
        'unfreeze_layers': 0,
        'learning_rate': 0.0005,
        'batch_size': 64,
        'epochs': 30,
        'patience': 10,
    },
    {
        # Partial: Adapts top dense blocks to medical domain
        'name': f'P1_DenseNet_partial_{WEIGHT_TAG}',
        'architecture': 'transfer',
        'base_name': 'densenet',
        'dropout': 0.5,
        'dense_units': 256,
        'unfreeze_layers': 50,
        'learning_rate': 0.0001,
        'batch_size': 64,
        'epochs': 30,
        'patience': 10,
    },
    {
        # Full: Dense connections enable stable full fine-tuning
        'name': f'P1_DenseNet_full_{WEIGHT_TAG}',
        'architecture': 'transfer',
        'base_name': 'densenet',
        'dropout': 0.5,
        'dense_units': 256,
        'unfreeze_layers': None,
        'learning_rate': 0.0001,
        'batch_size': 64,
        'epochs': 30,
        'patience': 10,
    },
]

for i, config in enumerate(configs_densenet, 1):
    result = train_experiment(config, data, datagen)
    phase1_results.append(result)
    save_to_drive(config['name'], DIR_PHASE_1)

In [ ]:
# ============================================================================
# PHASE 1 SUMMARY
print("\n" + "=" * 70)
print("📊 PHASE 1 COMPLETE - SUMMARY")
print("=" * 70)

summary_df = pd.DataFrame(phase1_results)
summary_df = summary_df.sort_values('best_val_f2', ascending=False)

print("\n🏆 RANKING BY F2 SCORE:\n")
print(summary_df[['name', 'best_val_f2', 'best_val_recall', 'best_val_precision', 'best_val_auc']].to_string(index=False))

# Best model
best_model = summary_df.iloc[0]
print(f"\n🥇 BEST MODEL: {best_model['name']}")
print(f"   F2 Score:  {best_model['best_val_f2']:.4f}")
print(f"   Recall:    {best_model['best_val_recall']:.4f}")
print(f"   Precision: {best_model['best_val_precision']:.4f}")
print(f"   AUC:       {best_model['best_val_auc']:.4f}")

summary_path = os.path.join(DIR_PHASE_1, 'phase1_summary.csv')
summary_df.to_csv(summary_path, index=False)
print(f"\n💾 Summary saved: {summary_path}")

print("\n" + "=" * 70)
print("➡️ NEXT: Phase 2 - Hyperparameter Tuning on Best Architecture")
print("=" * 70)

# Block 7: Phase 2 - Hyperparameter Tuning

## Goal
Fine-tune the Phase 1 winner to maximize validation F2/Recall.

## Winner Selection
Based on Phase 1 results:
- **If highest F2:** DenseNet_partial (F2=0.726)
- **If highest Recall:** CNN_shallow (Recall=97.45%, FN=5)

For clinical deployment, **CNN_shallow** is preferred due to:
1. Lowest false negatives (5 vs 21-29)
2. Highest recall (97.45%)
3. Appropriate model complexity (~50K params)

## Tuning Grid (for CNN)
| Parameter | Values |
|-----------|--------|
| filters_base | 32, 48, 64 |
| depth | 3, 4 |
| dropout | 0.3, 0.4, 0.5 |
| learning_rate | 0.001, 0.0005 |
| dense_units | 64, 128, 256 |

In [ ]:
# ============================================================================
# GOAL: Fine-tune the winning architecture from Phase 1
# Strategy:
# - If Transfer Learning won: fine-grained freeze depth tuning
# - If CNN won: depth, width, regularization tuning
# - Plus: dropout and learning rate experiments
# Total: 6 Runs

WINNER_ARCH, PHASE1_WEIGHT = get_phase1_winner()

if WINNER_ARCH is None:
    raise RuntimeError("❌ Phase 1 must be completed first!")

results = load_results()
phase1_winner = results['phase1']['best_model']

winner_config = phase1_winner.get('config', {})
winner_unfreeze = winner_config.get('unfreeze_layers', None)

# Determine freeze strategy from Phase 1
if winner_unfreeze == 0:
    winner_freeze_type = 'frozen'
elif winner_unfreeze is None:
    winner_freeze_type = 'full'
else:
    winner_freeze_type = 'partial'

WEIGHT_TAG = "W3"
data['class_weights'] = {0: 1.0, 1: 3.0}

print("="*70)
print("🔧 PHASE 2: HYPERPARAMETER TUNING")
print("="*70)
print(f"   Phase 1 Winner:    {phase1_winner['name']}")
print(f"   Architecture:      {WINNER_ARCH}")
print(f"   Freeze Strategy:   {winner_freeze_type}")
print(f"   Unfreeze Layers:   {winner_unfreeze}")
print(f"   Phase 1 Recall:    {phase1_winner['val_recall']:.4f}")
print(f"   Weight Strategy:   W3 (fixed)")
print("="*70)

phase2_results = []

# ============================================================================
# DEFINE TUNING CONFIGURATIONS

if WINNER_ARCH == 'custom_cnn':
    # ─────────────────────────────────────────────────────────────────────────
    # CNN TUNING: Architecture and regularization

    print("\n📦 CNN Hyperparameter Tuning")
    print("   Focus: depth, filters, dropout, learning rate")

    tuning_configs = [
        {
            'name': f'P2_CNN_Deeper_{WEIGHT_TAG}',
            'architecture': 'custom_cnn',
            'description': 'Deeper (5 blocks)',
            'filters_base': 64, 'depth': 5, 'dropout': 0.4, 'dense_units': 256,
            'learning_rate': 0.001, 'batch_size': 64, 'epochs': 50, 'patience': 15,
            'unfreeze_layers': None
        },
        {
            'name': f'P2_CNN_Wider_{WEIGHT_TAG}',
            'architecture': 'custom_cnn',
            'description': 'Wider (128 filters)',
            'filters_base': 128, 'depth': 4, 'dropout': 0.4, 'dense_units': 256,
            'learning_rate': 0.0005, 'batch_size': 32, 'epochs': 50, 'patience': 15,
            'unfreeze_layers': None
        },
        {
            'name': f'P2_CNN_HighDropout_{WEIGHT_TAG}',
            'architecture': 'custom_cnn',
            'description': 'High dropout (0.6)',
            'filters_base': 64, 'depth': 4, 'dropout': 0.6, 'dense_units': 256,
            'learning_rate': 0.001, 'batch_size': 64, 'epochs': 50, 'patience': 15,
            'unfreeze_layers': None
        },
        {
            'name': f'P2_CNN_LowLR_{WEIGHT_TAG}',
            'architecture': 'custom_cnn',
            'description': 'Low learning rate (0.0005)',
            'filters_base': 64, 'depth': 4, 'dropout': 0.4, 'dense_units': 256,
            'learning_rate': 0.0005, 'batch_size': 64, 'epochs': 50, 'patience': 15,
            'unfreeze_layers': None
        },
        {
            'name': f'P2_CNN_SmallBatch_{WEIGHT_TAG}',
            'architecture': 'custom_cnn',
            'description': 'Small batch (32)',
            'filters_base': 64, 'depth': 4, 'dropout': 0.4, 'dense_units': 256,
            'learning_rate': 0.001, 'batch_size': 32, 'epochs': 50, 'patience': 15,
            'unfreeze_layers': None
        },
        {
            'name': f'P2_CNN_BigDense_{WEIGHT_TAG}',
            'architecture': 'custom_cnn',
            'description': 'Larger dense layer (512)',
            'filters_base': 64, 'depth': 4, 'dropout': 0.5, 'dense_units': 512,
            'learning_rate': 0.001, 'batch_size': 64, 'epochs': 50, 'patience': 15,
            'unfreeze_layers': None
        },
    ]

else:
    # ─────────────────────────────────────────────────────────────────────────
    # TRANSFER LEARNING TUNING: Fine-grained freeze depth

    if WINNER_ARCH == 'resnet50':
        total_layers = 175
        # Test around Phase 1 winner's value
        if winner_freeze_type == 'frozen':
            unfreeze_options = [20, 40, 60]  # Start unfreezing
        elif winner_freeze_type == 'partial':
            base = winner_unfreeze if winner_unfreeze else 80
            unfreeze_options = [base-20, base, base+20, base+40]
        else:  # full
            unfreeze_options = [100, 120, 140, None]  # Around full

    elif WINNER_ARCH == 'vgg16':
        total_layers = 19
        if winner_freeze_type == 'frozen':
            unfreeze_options = [4, 8, 12]
        elif winner_freeze_type == 'partial':
            base = winner_unfreeze if winner_unfreeze else 8
            unfreeze_options = [max(4, base-4), base, min(19, base+4)]
        else:  # full
            unfreeze_options = [12, 15, None]

    else:  # efficientnet
        total_layers = 237
        if winner_freeze_type == 'frozen':
            unfreeze_options = [40, 80, 120]
        elif winner_freeze_type == 'partial':
            base = winner_unfreeze if winner_unfreeze else 100
            unfreeze_options = [base-30, base, base+30, base+60]
        else:  # full
            unfreeze_options = [150, 180, 200, None]

    # Remove duplicates and None handling
    unfreeze_options = list(dict.fromkeys(unfreeze_options))

    print(f"\n📦 {WINNER_ARCH.upper()} Hyperparameter Tuning")
    print(f"   Total layers: {total_layers}")
    print(f"   Phase 1 unfreeze: {winner_unfreeze}")
    print(f"   Testing unfreeze: {unfreeze_options}")

    tuning_configs = []

    # Freeze depth experiments
    for unfreeze in unfreeze_options:
        if unfreeze is None:
            name_suffix = "Full"
            desc = f"Full fine-tuning (all {total_layers} layers)"
            lr = 0.00005
        else:
            name_suffix = f"Unfreeze{unfreeze}"
            desc = f"Unfreeze {unfreeze}/{total_layers} layers ({100*unfreeze/total_layers:.0f}%)"
            lr = 0.0001

        tuning_configs.append({
            'name': f'P2_{WINNER_ARCH}_{name_suffix}_{WEIGHT_TAG}',
            'architecture': WINNER_ARCH,
            'description': desc,
            'dropout': 0.5,
            'dense_units': 256,
            'unfreeze_layers': unfreeze,
            'learning_rate': lr,
            'batch_size': 32,
            'epochs': 50,
            'patience': 15
        })

    # Add regularization experiments
    best_unfreeze = winner_unfreeze if winner_unfreeze else unfreeze_options[len(unfreeze_options)//2]

    tuning_configs.extend([
        {
            'name': f'P2_{WINNER_ARCH}_HighDropout_{WEIGHT_TAG}',
            'architecture': WINNER_ARCH,
            'description': f'High dropout (0.7) + {best_unfreeze} layers',
            'dropout': 0.7,
            'dense_units': 256,
            'unfreeze_layers': best_unfreeze,
            'learning_rate': 0.0001,
            'batch_size': 32,
            'epochs': 50,
            'patience': 15
        },
        {
            'name': f'P2_{WINNER_ARCH}_VeryLowLR_{WEIGHT_TAG}',
            'architecture': WINNER_ARCH,
            'description': f'Very low LR (0.00003) + {best_unfreeze} layers',
            'dropout': 0.5,
            'dense_units': 256,
            'unfreeze_layers': best_unfreeze,
            'learning_rate': 0.00003,
            'batch_size': 32,
            'epochs': 50,
            'patience': 15
        },
    ])

# ============================================================================
# RUN TUNING EXPERIMENTS

print(f"\n🔧 Running {len(tuning_configs)} experiments...")

for i, config in enumerate(tuning_configs, 1):
    print(f"\n{'='*70}")
    print(f"▶️ [{i}/{len(tuning_configs)}] {config['description']}")
    print(f"{'='*70}")

    result = train_experiment(config, data, datagen)
    phase2_results.append(result)
    save_to_drive(config['name'], DIR_PHASE_2)

# ============================================================================
# COMPARE RESULTS

print("\n" + "="*70)
print("📊 PHASE 2 RESULTS (vs Phase 1 Winner)")
print("="*70)

if phase2_results:
    comparison_data = []

    # Add Phase 1 winner as baseline
    comparison_data.append({
        'Name': phase1_winner['name'],
        'Description': '⭐ Phase 1 Winner (baseline)',
        'Unfreeze': winner_unfreeze,
        'Val_Recall': phase1_winner['val_recall'],
        'Val_AUC': phase1_winner.get('val_auc', 0),
        'Source': 'P1'
    })

    # Add Phase 2 results
    for res in phase2_results:
        if 'error' in res:
            print(f"⚠️ Skipping {res['config']['name']} (failed)")
            continue

        comparison_data.append({
            'Name': res['config']['name'],
            'Description': res['config'].get('description', ''),
            'Unfreeze': res['config'].get('unfreeze_layers', 'N/A'),
            'Val_Recall': res.get('best_val_recall', 0),
            'Val_AUC': res.get('val_auc', 0),
            'Source': 'P2'
        })

    df_compare = pd.DataFrame(comparison_data)
    df_compare = df_compare.sort_values('Val_Recall', ascending=False)

    print("\n")
    print(df_compare.to_string(index=False))

    # Find overall best
    best_idx = df_compare['Val_Recall'].idxmax()
    best_row = df_compare.loc[best_idx]

    # Calculate improvement
    p1_recall = phase1_winner['val_recall']
    p2_recalls = df_compare[df_compare['Source'] == 'P2']['Val_Recall']
    p2_best_recall = p2_recalls.max() if len(p2_recalls) > 0 else 0
    improvement = p2_best_recall - p1_recall

    print("\n" + "="*70)
    if improvement > 0:
        print(f"✅ Phase 2 IMPROVED over Phase 1: +{improvement:.4f}")
    elif improvement == 0:
        print(f"⚖️  Phase 2 matched Phase 1 (no improvement)")
    else:
        print(f"⚠️  Phase 2 did NOT improve: {improvement:.4f}")
    print("="*70)

    print("\n" + "="*70)
    print("🏆 OVERALL BEST MODEL")
    print("="*70)
    print(f"   Model:       {best_row['Name']}")
    print(f"   Source:      Phase {best_row['Source'][-1]}")
    print(f"   Unfreeze:    {best_row['Unfreeze']}")
    print(f"   Val Recall:  {best_row['Val_Recall']:.4f} ← PRIMARY METRIC")
    print(f"   Val AUC:     {best_row['Val_AUC']:.4f}")
    print("="*70)

    update_phase2_results(phase2_results, WEIGHT_TAG)

    csv_path = os.path.join(BASE_PATH, f'phase2_results_{WEIGHT_TAG}.csv')
    df_compare.to_csv(csv_path, index=False)
    print(f"\n💾 Results saved: {csv_path}")
    print(f"💾 Models saved:  {DIR_PHASE_2}")

    print("\n" + "="*70)
    print("✅ PHASE 2 COMPLETE")
    print("="*70)
    print(f"   → Proceed to Block 8 (Phase 3: Final Evaluation)")
    print("="*70)

else:
    print("⚠️ No results available. Check for errors above.")

# Block 8: Phase 3 - Final Evaluation

## Goal
Unbiased evaluation on the held-out **TEST set** (used only once!).

## Pipeline

| Step | Action | Data Split |
|------|--------|------------|
| 1 | Load best model from Phase 2 | - |
| 2 | Threshold calibration (optional) | Validation |
| 3 | Final predictions | **Test** |
| 4 | Compute confusion matrix | **Test** |
| 5 | Clinical interpretation | - |

## Threshold Calibration
Default threshold: 0.5
Can be adjusted to optimize Recall vs Precision trade-off:
- Lower threshold → Higher Recall, More FP
- Higher threshold → Lower Recall, Fewer FP

## Final Metrics
- Recall (primary): % of malignant cases detected
- F2 Score: Weighted harmonic mean
- Confusion Matrix: TP, FP, TN, FN
- Clinical interpretation: Missed cancers vs unnecessary biopsies

In [ ]:
# ============================================================================
# GOAL: Final unbiased evaluation on TEST set
# Steps:

results = load_results()

if not results['phase1']['completed']:
    raise RuntimeError("❌ Phase 1 must be completed first!")

# Phase 1 best
p1_best = results['phase1']['best_model']
p1_name = p1_best['name']
p1_recall = p1_best['val_recall']

print("="*70)
print("🏁 PHASE 3: FINAL EVALUATION")
print("="*70)

print(f"\n📊 PHASE COMPARISON:")
print("-"*50)
print(f"   Phase 1 Best: {p1_name}")
print(f"   Val Recall:   {p1_recall:.4f}")

# Check Phase 2
if results['phase2']['completed']:
    p2_best = results['phase2']['best_model']
    p2_name = p2_best['name']
    p2_recall = p2_best['val_recall']

    print(f"\n   Phase 2 Best: {p2_name}")
    print(f"   Val Recall:   {p2_recall:.4f}")

    improvement = p2_recall - p1_recall
    print(f"\n   Improvement:  {improvement:+.4f}")

    # Auto-select best
    if p2_recall >= p1_recall:
        FINAL_MODEL_NAME = p2_name
        FINAL_MODEL_RECALL = p2_recall
        SOURCE_PHASE = 2
        SOURCE_DIR = DIR_PHASE_2
        print(f"\n   ✅ Using Phase 2 model (better or equal)")
    else:
        FINAL_MODEL_NAME = p1_name
        FINAL_MODEL_RECALL = p1_recall
        SOURCE_PHASE = 1
        SOURCE_DIR = DIR_PHASE_1
        print(f"\n   ⚠️ Using Phase 1 model (Phase 2 didn't improve)")
else:
    print(f"\n   Phase 2: Not completed")
    FINAL_MODEL_NAME = p1_name
    FINAL_MODEL_RECALL = p1_recall
    SOURCE_PHASE = 1
    SOURCE_DIR = DIR_PHASE_1
    print(f"\n   → Using Phase 1 model")

print("-"*50)

# ┌─────────────────────────────────────────────────────────────────────────────┐
# └─────────────────────────────────────────────────────────────────────────────┘
WEIGHT_TAG = FINAL_MODEL_NAME.split('_')[-1]
final_model_path = os.path.join(SOURCE_DIR, f'{FINAL_MODEL_NAME}.keras')

print(f"\n📥 LOADING MODEL:")
print(f"   Name:   {FINAL_MODEL_NAME}")
print(f"   Source: Phase {SOURCE_PHASE}")
print(f"   Path:   {final_model_path}")

if not os.path.exists(final_model_path):
    raise FileNotFoundError(f"❌ Model not found: {final_model_path}")

final_model = tf.keras.models.load_model(final_model_path)
print("   ✅ Model loaded successfully")

print("\n" + "="*70)
print("🎯 THRESHOLD CALIBRATION")
print("="*70)
print("   Dataset:    VALIDATION (never test set!)")
print("   Goal:       Maximize Recall while keeping Precision ≥ 40%")
print("-"*70)

val_probs = final_model.predict(data['x_val'], verbose=0)
val_true = np.argmax(data['y_val'], axis=1)
val_scores = val_probs[:, 1]

thresholds = np.arange(0.10, 0.90, 0.05)
best_threshold = 0.5
best_recall = 0.0
min_precision = 0.40

print(f"\n   {'Threshold':<12} {'Recall':<10} {'Precision':<10} {'Status'}")
print(f"   {'-'*45}")

for t in thresholds:
    preds = (val_scores >= t).astype(int)
    rec = recall_score(val_true, preds)
    prec = precision_score(val_true, preds, zero_division=0)

    if prec < min_precision:
        status = "❌ Low precision"
    elif rec > best_recall:
        status = "✅ NEW BEST"
        best_recall = rec
        best_threshold = t
    else:
        status = ""

    print(f"   {t:<12.2f} {rec:<10.4f} {prec:<10.4f} {status}")

print(f"\n   ✅ Optimal Threshold: {best_threshold:.2f}")
print(f"      Expected Recall:  {best_recall:.4f}")

print("\n" + "="*70)
print("🧪 FINAL TEST EVALUATION")
print("="*70)
print("   ⚠️  TEST SET - Used only ONCE for final, unbiased evaluation")
print("-"*70)

test_probs = final_model.predict(data['x_test'], verbose=0)
test_true = np.argmax(data['y_test'], axis=1)
test_scores = test_probs[:, 1]
test_preds = (test_scores >= best_threshold).astype(int)

# Metrics
final_recall = recall_score(test_true, test_preds)
final_precision = precision_score(test_true, test_preds)
final_f1 = f1_score(test_true, test_preds)
final_auc = roc_auc_score(test_true, test_scores)
cm = confusion_matrix(test_true, test_preds)
tn, fp, fn, tp = cm.ravel()

print("\n" + "="*70)
print(f"🏆 FINAL RESULTS: {FINAL_MODEL_NAME}")
print("="*70)

print(f"""
   ┌─────────────────────────────────────────────────────────────┐
   │  PRIMARY METRIC                                             │
   │  ─────────────────────────────────────────────────────────  │
   │  Recall (Sensitivity):  {final_recall:.4f}  ({100*final_recall:.1f}%)               │
   │                                                             │
   │  SECONDARY METRICS                                          │
   │  ─────────────────────────────────────────────────────────  │
   │  Precision:             {final_precision:.4f}  ({100*final_precision:.1f}%)               │
   │  F1 Score:              {final_f1:.4f}                           │
   │  AUC:                   {final_auc:.4f}                           │
   │  Threshold:             {best_threshold:.2f}                             │
   └─────────────────────────────────────────────────────────────┘
""")

print("   🎯 CONFUSION MATRIX:")
print(f"""
                        Predicted
                     Benign    Malignant
                   ┌─────────┬─────────┐
   Actual Benign   │  {tn:5d}  │  {fp:5d}  │
                   ├─────────┼─────────┤
   Actual Malig.   │  {fn:5d}  │  {tp:5d}  │
                   └─────────┴─────────┘
""")

total_malignant = fn + tp
total_benign = tn + fp

print("   🏥 CLINICAL INTERPRETATION:")
print(f"""
   Malignant Cases: {total_malignant}
   ├── ✅ Detected (True Positive):   {tp:4d}  ({100*tp/total_malignant:5.1f}%)
   └── ❌ Missed (False Negative):    {fn:4d}  ({100*fn/total_malignant:5.1f}%)  ← DANGEROUS

   Benign Cases: {total_benign}
   ├── ✅ Correct (True Negative):    {tn:4d}  ({100*tn/total_benign:5.1f}%)
   └── ⚠️  False Alarm (False Pos):   {fp:4d}  ({100*fp/total_benign:5.1f}%)  → unnecessary biopsy
""")

cm_path = os.path.join(DIR_PHASE_3, f'FINAL_confusion_matrix.png')
plot_confusion_matrix(cm, f'FINAL: {FINAL_MODEL_NAME}', save_path=cm_path)

final_results = {
    'model_name': FINAL_MODEL_NAME,
    'source_phase': SOURCE_PHASE,
    'weight_strategy': WEIGHT_TAG,
    'threshold': best_threshold,
    'test_recall': final_recall,
    'test_precision': final_precision,
    'test_f1': final_f1,
    'test_auc': final_auc,
    'true_neg': int(tn),
    'false_pos': int(fp),
    'false_neg': int(fn),
    'true_pos': int(tp),
    'total_malignant': int(total_malignant),
    'total_benign': int(total_benign),
    'missed_cancers': int(fn),
    'detection_rate': float(tp/total_malignant)
}

# Update tracking
results['phase3']['completed'] = True
results['phase3']['final_results'] = final_results
save_results(results)

df_final = pd.DataFrame([final_results])
csv_path = os.path.join(DIR_PHASE_3, f'FINAL_RESULTS.csv')
df_final.to_csv(csv_path, index=False)

# Copy model
model_dest = os.path.join(DIR_PHASE_3, f'FINAL_MODEL.keras')
shutil.copy(final_model_path, model_dest)

print(f"\n💾 FILES SAVED:")
print(f"   Results:  {csv_path}")
print(f"   Model:    {model_dest}")
print(f"   CM Plot:  {cm_path}")

print("\n" + "="*70)
print("✅ ALL EXPERIMENTS COMPLETE!")
print("="*70)

show_experiment_status()

print("\n" + "="*70)
print("📋 FINAL SUMMARY")
print("="*70)
print(f"""
   Model:           {FINAL_MODEL_NAME}
   Source:          Phase {SOURCE_PHASE}

   Test Recall:     {final_recall:.4f} ({100*final_recall:.1f}%)
   Test Precision:  {final_precision:.4f} ({100*final_precision:.1f}%)
   Test AUC:        {final_auc:.4f}

   Missed Cancers:  {fn} out of {total_malignant} ({100*fn/total_malignant:.1f}%)
""")
print("="*70)